In [9]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns=None
from pprint import pprint
import seaborn as sns
from sklearn.metrics import roc_auc_score,auc,roc_curve
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler


# Decision Tree Classifier without Scaling

In [4]:
german_df=pd.read_csv('GermanCredit.csv')
german_df.head()
german_df.isnull().sum()[german_df.isnull().sum()!=0]#No missing values
list(german_df.columns)
german_df.dtypes.value_counts()
german_df.Class.value_counts()
y=german_df.Class
x=german_df.drop('Class',axis=1)
print(x.shape,y.shape)
y=y.map({'Good':0,'Bad':1})
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
clf=DecisionTreeClassifier()
# clf.fit(x_train,y_train)
from sklearn.metrics import classification_report,confusion_matrix,precision_score,recall_score
clf.fit(x_train,y_train)
print('Train Classification Report: ')
print(classification_report(y_train,clf.predict(x_train)))
print('Train confusion Matrix')
print(pd.crosstab(y_train,clf.predict(x_train),rownames=['Actual'],colnames=['Predicted']))
print('Test Confusion Matrix')
print(pd.crosstab(y_test,clf.predict(x_test),colnames=['Predicted'],rownames=['Actual']))
print('Test Classification Report')
print(classification_report(y_test,clf.predict(x_test)))

(1000, 61) (1000,)
Train Classification Report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       491
           1       1.00      1.00      1.00       209

   micro avg       1.00      1.00      1.00       700
   macro avg       1.00      1.00      1.00       700
weighted avg       1.00      1.00      1.00       700

Train confusion Matrix
Predicted    0    1
Actual             
0          491    0
1            0  209
Test Confusion Matrix
Predicted    0   1
Actual            
0          149  60
1           46  45
Test Classification Report
              precision    recall  f1-score   support

           0       0.76      0.71      0.74       209
           1       0.43      0.49      0.46        91

   micro avg       0.65      0.65      0.65       300
   macro avg       0.60      0.60      0.60       300
weighted avg       0.66      0.65      0.65       300



# Decison Tree with Scaling 

In [26]:
german_df=pd.read_csv('GermanCredit.csv')
german_df.head()
german_df.isnull().sum()[german_df.isnull().sum()!=0]#No missing values
list(german_df.columns)
german_df.dtypes.value_counts()
german_df.Class.value_counts()
y=german_df.Class
x=german_df.drop('Class',axis=1)
print(x.shape,y.shape)
y=y.map({'Good':0,'Bad':1})
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
scaler=StandardScaler()
x_train.iloc[:,0:7]=scaler.fit_transform(x_train.iloc[:,0:7])
x_test.iloc[:,0:7]=scaler.transform(x_test.iloc[:,0:7])
clf=DecisionTreeClassifier()
clf.fit(x_train,y_train)
from sklearn.metrics import classification_report,confusion_matrix,precision_score,recall_score
clf.fit(x_train,y_train)
print('Train Classification Report: ')
print(classification_report(y_train,clf.predict(x_train)))
print('Train confusion Matrix')
print(pd.crosstab(y_train,clf.predict(x_train),rownames=['Actual'],colnames=['Predicted']))
print('Test Confusion Matrix')
print(pd.crosstab(y_test,clf.predict(x_test),colnames=['Predicted'],rownames=['Actual']))
print('Test Classification Report')
print(classification_report(y_test,clf.predict(x_test)))

(1000, 61) (1000,)


C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\Dell\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  
C:\Users\Dell\Anaconda3\lib\site-packages\pand

Train Classification Report: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       491
           1       1.00      1.00      1.00       209

   micro avg       1.00      1.00      1.00       700
   macro avg       1.00      1.00      1.00       700
weighted avg       1.00      1.00      1.00       700

Train confusion Matrix
Predicted    0    1
Actual             
0          491    0
1            0  209
Test Confusion Matrix
Predicted    0   1
Actual            
0          155  54
1           44  47
Test Classification Report
              precision    recall  f1-score   support

           0       0.78      0.74      0.76       209
           1       0.47      0.52      0.49        91

   micro avg       0.67      0.67      0.67       300
   macro avg       0.62      0.63      0.62       300
weighted avg       0.68      0.67      0.68       300



# With Scaling Turns out to be better

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.neural_network import MLPClassifier

In [30]:
log_clf = LogisticRegression(solver="liblinear", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=10, random_state=42)
svm_clf = SVC(gamma="auto", random_state=42,probability=True)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='soft')
voting_clf.fit(x_train, y_train)
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.7733333333333333
RandomForestClassifier 0.78
SVC 0.75
VotingClassifier 0.7766666666666666


# Finding the optimal number of trees by using Staged Predict Method

In [35]:
from sklearn.ensemble import GradientBoostingClassifier
clf=GradientBoostingClassifier()
clf.fit(x_train,y_train)
from sklearn.metrics import f1_score
f1_scores=[f1_score(y_test,y_val) for y_val in clf.staged_predict(x_test)]
np.argmax(f1_scores)#Staged predict gives predictions at each stage.

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetri

87

# It is also possible to early stopping by stopping training at optimal number instead of training for large predictors and finding the optimal number


In [ ]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True, random_state=42)

min_val_error = float("inf")
error_going_up = 0
for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0
    else:
        error_going_up += 1
        if error_going_up == 5:
            break  # early stopping

# MLP CLassifier

In [38]:
mlp=MLPClassifier()
mlp.fit(x_train,y_train)
print('Train Classification Report: ')
print(classification_report(y_train,clf.predict(x_train)))
print('Train confusion Matrix')
print(pd.crosstab(y_train,clf.predict(x_train),rownames=['Actual'],colnames=['Predicted']))
print('Test Confusion Matrix')
print(pd.crosstab(y_test,clf.predict(x_test),colnames=['Predicted'],rownames=['Actual']))
print('Test Classification Report')
print(classification_report(y_test,clf.predict(x_test)))

Train Classification Report: 
              precision    recall  f1-score   support

           0       0.91      0.99      0.95       491
           1       0.96      0.77      0.86       209

   micro avg       0.92      0.92      0.92       700
   macro avg       0.94      0.88      0.90       700
weighted avg       0.93      0.92      0.92       700

Train confusion Matrix
Predicted    0    1
Actual             
0          485    6
1           48  161
Test Confusion Matrix
Predicted    0   1
Actual            
0          191  18
1           49  42
Test Classification Report
              precision    recall  f1-score   support

           0       0.80      0.91      0.85       209
           1       0.70      0.46      0.56        91

   micro avg       0.78      0.78      0.78       300
   macro avg       0.75      0.69      0.70       300
weighted avg       0.77      0.78      0.76       300



C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# MLP is performing better 